# 과제 1
** 고객 세분화 및 RFM 분석**

- 브라질 지역별 고객들의 구매 패턴을 분석하여 RFM(Recency, Frequency, Monetary) 모델을 구축하고, 고객을 세분화하여 각 세그먼트의 특성과 비즈니스 전략을 제시하세요.
1. 이상치/ 결측치 제거 
2. 데이터 통합 (고객 기본 정보, 구매 상품 정보, 결제 정보, 고객위치정보)
3. 파생변수 생성 RFM 지표 계산 (RFM 지표계산)


# 데이터 살펴보기 



# 브라질 고객 구매 패턴 분석
- 필요한 데이터 셋 (orders, olist_order_items_datasat) order_id를 공유한다. 

In [10]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# 불러올 파일 목록
files = [
    "olist_orders_dataset.csv",
    "olist_customers_dataset.csv",
    "olist_order_items_dataset.csv",
    "olist_products_dataset.csv",
    "olist_sellers_dataset.csv",
    "olist_geolocation_dataset.csv",
    "olist_order_payments_dataset.csv",
    "olist_order_reviews_dataset.csv"
]

# 여러 개 파일을 데이터프레임으로 불러오기
dfs = {}
for f in files:
    dfs[f] = kagglehub.load_dataset(
        KaggleDatasetAdapter.PANDAS,
        "olistbr/brazilian-ecommerce",
        f
    )

# 예시: orders 데이터 확인
print(dfs["olist_orders_dataset.csv"].head())

/var/folders/98/vj3q9b254k79y681ng12dddh0000gn/T/ipykernel_10248/1011438912.py:19: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  dfs[f] = kagglehub.load_dataset(
/var/folders/98/vj3q9b254k79y681ng12dddh0000gn/T/ipykernel_10248/1011438912.py:19: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  dfs[f] = kagglehub.load_dataset(
/var/folders/98/vj3q9b254k79y681ng12dddh0000gn/T/ipykernel_10248/1011438912.py:19: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  dfs[f] = kagglehub.load_dataset(
/var/folders/98/vj3q9b254k79y681ng12dddh0000gn/T/ipykernel_10248/1011438912.py:19: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  dfs[f] = kagglehub.load_dataset(
/var/folders/98/vj3q9b254k79y681ng12dddh0000gn/T/ipykernel_10248/1011438912.py:19: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  dfs

                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00           2018-08

In [12]:
orders = dfs["olist_orders_dataset.csv"]
order_items = dfs["olist_order_items_dataset.csv"]
products = dfs["olist_products_dataset.csv"]
sellers = dfs["olist_sellers_dataset.csv"]
geolocation = dfs["olist_geolocation_dataset.csv"]
order_payments = dfs["olist_order_payments_dataset.csv"]
olist_order_reviews = dfs["olist_order_reviews_dataset.csv"]
customers = dfs['olist_customers_dataset.csv']
print(orders.tail())


                               order_id                       customer_id  \
99436  9c5dedf39a927c1b2549525ed64a053c  39bd1228ee8140590ac3aca26f2dfe00   
99437  63943bddc261676b46f01ca7ac2f7bd8  1fca14ff2861355f6e5f14306ff977a7   
99438  83c1379a015df1e13d02aae0204711ab  1aa71eb042121263aafbe80c1b562c9c   
99439  11c177c8e97725db2631073c19f07b62  b331b74b18dc79bcdf6532d51e1637c1   
99440  66dea50a8b16d9b4dee7af250b4be1a5  edb027a75a1449115f6b43211ae02a24   

      order_status order_purchase_timestamp    order_approved_at  \
99436    delivered      2017-03-09 09:54:05  2017-03-09 09:54:05   
99437    delivered      2018-02-06 12:58:58  2018-02-06 13:10:37   
99438    delivered      2017-08-27 14:46:43  2017-08-27 15:04:16   
99439    delivered      2018-01-08 21:28:27  2018-01-08 21:36:21   
99440    delivered      2018-03-08 20:57:30  2018-03-09 11:20:28   

      order_delivered_carrier_date order_delivered_customer_date  \
99436          2017-03-10 11:18:03           2017-03-17 15:0

In [19]:
# 상품 단위 집계
order_items_products = order_items.merge(products, on='product_id', how='left')
product_summary = order_items_products.groupby(['product_id','product_category_name']).agg({
    'order_id': 'nunique',        # 판매된 주문 수
    'price': ['sum','mean'],      # 총 매출, 평균 가격
    'freight_value': 'mean'       # 평균 배송비
}).reset_index()

# 컬럼 이름 정리
product_summary.columns = ['product_id','category','num_orders','total_sales','avg_price','avg_freight']

# 상위 10개 상품 확인
print(product_summary.sort_values('total_sales', ascending=False).head(10))


                             product_id                category  num_orders  \
23650  bb50f2e236e5eea0100680137654686c            beleza_saude         187   
13813  6cdd53843498f92890544667809f1595            beleza_saude         151   
27114  d6160fb7873f184099d9bc95e30376af                     pcs          35   
26545  d1c427060a0f73f6b889a5c7c61f2ac4  informatica_acessorios         323   
19394  99a4788cb24856965c36a24e339b6058         cama_mesa_banho         467   
7905   3dd2a17168ec895c781a9191c1e95ad7  informatica_acessorios         255   
4904   25c38557cf793876c5abdd5931f922db                   bebes          38   
12129  5f504b3a1c75b73d6151be81eb05bdc9              cool_stuff          63   
10665  53b36df67ebb7c41585e8d54d6772e08      relogios_presentes         306   
21724  aca2eb7d00ea1a7b8ebd4e68314663af        moveis_decoracao         431   

       total_sales    avg_price  avg_freight  
23650     63885.00   327.615385    19.082564  
13813     54730.20   350.834615    2

## 데이터 전처리 

In [14]:
import pandas as pd
import numpy as np

# 1️⃣ 데이터 불러오기 (이미 dfs 딕셔너리에 있다고 가정)
orders = dfs["olist_orders_dataset.csv"]
order_items = dfs["olist_order_items_dataset.csv"]
payments = dfs["olist_order_payments_dataset.csv"]
customers = dfs["olist_customers_dataset.csv"]

# # 2️⃣ 주문 데이터: delivered 주문만 사용 + 날짜 처리
orders = orders[orders['order_status'] == 'delivered'].copy()
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

# 스냅샷 기준일(마지막 주문일 + 1일)
snapshot_date = orders['order_purchase_timestamp'].max() + pd.Timedelta(days=1)

# # 3️⃣ 주문별 금액 계산 (가격 + 배송비)
order_items['total_amount'] = order_items['price'] + order_items['freight_value']
order_amounts = order_items.groupby('order_id')['total_amount'].sum().reset_index()

# 4️⃣ 주문별 결제 금액 집계
order_payments = payments.groupby('order_id')['payment_value'].sum().reset_index()

# # 5️⃣ 주문 데이터와 결합 (금액 추가)
df = orders.merge(order_amounts, on='order_id', how='left') \
           .merge(order_payments, on='order_id', how='left')

# # 6️⃣ 고객 정보 결합
df = df.merge(customers[['customer_id', 'customer_unique_id']], on='customer_id', how='left')

# # 7️⃣ 결측치 제거
df = df.dropna(subset=['customer_unique_id', 'order_purchase_timestamp', 'payment_value'])

# # 8️⃣ 고객별 RFM 집계
rfm = df.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (snapshot_date - x.max()).days,  # Recency
    'order_id': 'nunique',          # Frequency
    'payment_value': 'sum'          # Monetary (실제 결제금액 기준)
}).reset_index()

# # 9️⃣ 컬럼명 변경
rfm.rename(columns={
    'order_purchase_timestamp': 'Recency',
    'order_id': 'Frequency',
    'payment_value': 'Monetary'
}, inplace=True)

# # 10️⃣ 최종 RFM 데이터 확인
print(rfm.tail())


                     customer_unique_id  Recency  Frequency  Monetary
93352  fffcf5a5ff07b0908bd4e2dbc735a684      447          1   2067.42
93353  fffea47cd6d3cc0a88bd621562a9d061      262          1     84.58
93354  ffff371b4d645b6ecea244b27531430a      568          1    112.46
93355  ffff5962728ec6157033ef9805bacc48      119          1    133.69
93356  ffffd2657e2aad2907e67c3e9daecbeb      484          1     71.56


In [15]:
#결측치/ 이상치 확인 
import pandas as pd

# 예시: orders, customers, payments 데이터프레임 사용
orders = dfs["olist_orders_dataset.csv"]
customers = dfs["olist_customers_dataset.csv"]
payments = dfs["olist_order_payments_dataset.csv"]

# ✅ 결측치 비율 확인
print("=== 결측치 비율 ===")
print(orders.isnull().mean().sort_values(ascending=False))

# ✅ orders 테이블 결측치 처리
# 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date' 등 일부 컬럼에 결측치가 있음
# 주문 상태(order_status)가 'delivered'인 데이터만 남기는 게 보통 좋음
orders = orders[orders["order_status"] == "delivered"]

# ✅ customers 테이블 결측치 제거 (customer_id 필수)
customers = customers.dropna(subset=["customer_id"])

# ✅ payments 테이블에서 결측치 처리
payments = payments.dropna(subset=["payment_value"])  # 결제 금액이 없는 건 제거

=== 결측치 비율 ===
order_delivered_customer_date    0.029817
order_delivered_carrier_date     0.017930
order_approved_at                0.001609
order_id                         0.000000
customer_id                      0.000000
order_status                     0.000000
order_purchase_timestamp         0.000000
order_estimated_delivery_date    0.000000
dtype: float64


## RFM 점수화(고객 세그먼트 분류)

In [16]:
# Recency 점수 (값이 낮을수록 높은 점수)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1]).astype(int)

# Frequency 점수 (값이 클수록 높은 점수)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

# Monetary 점수 (값이 클수록 높은 점수)
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

# 최종 RFM Score
rfm['RFM_Score'] = rfm['R_Score'] + rfm['F_Score'] + rfm['M_Score']

# 세그먼트 분류 
def segment_customer(row):
    if row['RFM_Score'] >= 13:
        return '최우수 고객 (Champions)'
    elif 10 <= row['RFM_Score'] < 13:
        return '충성 고객 (Loyal Customers)'
    elif 7 <= row['RFM_Score'] < 10:
        return '잠재 고객 (Potential Loyalists)'
    elif 5 <= row['RFM_Score'] < 7:
        return '이탈 위험 고객 (At Risk)'
    else:
        return '휴면 고객 (Lost Customers)'

rfm['Segment'] = rfm.apply(segment_customer, axis=1)

# 세그먼트 별 분포 확인 
segment_summary = rfm['Segment'].value_counts(normalize=True).mul(100).round(2)
print(segment_summary)


Segment
잠재 고객 (Potential Loyalists)    41.12
충성 고객 (Loyal Customers)        33.75
이탈 위험 고객 (At Risk)             13.14
최우수 고객 (Champions)              8.60
휴면 고객 (Lost Customers)          3.39
Name: proportion, dtype: float64


## 지역별 분석 
- 고객데이터에 지역 정보 결합 

In [ ]:
ㅊ

   customer_state     Recency  Frequency    Monetary
0              PB  246.178503   1.072937  279.981190
1              AC  307.700000   1.100000  267.882625
2              RO  273.131148   1.135246  254.384836
3              AL  266.052500   1.050000  252.464175
4              AP  245.910448   1.029851  249.964776
5              RR  269.195122   1.048780  239.091951
6              PI  249.683438   1.058700  232.122348
7              PA  264.252376   1.054910  231.766463
8              TO  248.043796   1.051095  227.727518
9              RN  250.884211   1.042105  219.764842
10             SE  271.465875   1.044510  219.316291
11             MA  260.818308   1.070735  217.046269
12             MT  244.344244   1.074492  214.843499
13             CE  252.928181   1.035909  213.577799
14             AM  265.082759   1.110345  211.102069
15             PE  237.370138   1.089711  209.241844
16             MS  234.549858   1.066952  200.566439
17             BA  242.058571   1.068691  193.

In [ ]:
# customers 컬럼 확인
print(customers.columns)

# 샘플 데이터 확인
print(customers.head())

# 인사이트 도출 및 비즈니스 전략 
1. 인사이트 도출 
    **최우수 고객이 많은 지역** 
    - SP, RS(상파울루주와 히우그란지두술주가 ): 프리미엄 상품과 맞춤형 마케팅에 적합하다. 
    - 평균 Monetary 높으므로 할인보다는 혜택 중심의 로열티 프로그램 추천

    **잠재 고객이 많은 지역**
    - BA, MG(바이아주,미나스제라이스주): 첫 구매 이후 재구매 유도 필요
    - 이메일, 앱푸시, 할인 쿠폰 등 재구매 캠페인 강화 

    **이탈 위험 / 휴면 고객이 많은 지역**
    - BA,RJ(바이아주.리우데자네이루주): 장기 미구매 고객 집중 관리 필요 
    - 맞춤형 프로모션, 한정판 상품 개인화 추천 등의 재참여 유도 

    **지역별 구매력 차이** 
    - SP. RS: 고가 상품 판매 가능 
    - BA, MG : 가격 민감도 높음-> 소규모 할인 프로모션 효과적임 

2. 비즈니스 전략 

    A. VIP/충성 고객 전략
        •	전용 혜택 제공 (포인트 적립, 무료 배송, 프리미엄 서비스)
        •	조기 신상품 안내, 한정판 이벤트

    B. 잠재 고객 전략
        •	첫 구매 이후 7~30일 내 재구매 유도 쿠폰 발송
        •	관심 상품 기반 추천 상품 마케팅

    C. 이탈 위험/휴면 고객 전략
        •	이메일, SMS, 앱 푸시로 맞춤형 할인·프로모션 제공
        •	휴면 고객용 리타겟팅 광고 진행

    D. 지역 맞춤 전략
        •	SP, RJ → 고가/프리미엄 상품 중심 캠페인
        •	BA, MG → 가격 민감도 고려한 할인/쿠폰 마케팅

    E. 추가 전략
        •	RFM 점수 추적 → 주기적으로 RFM 갱신, 세그먼트 이동 감지
        •	고객 Lifetime Value 예측 → 높은 LTV 고객 우선 집중
        •	지역별 물류 최적화 → 배송비 및 배송 속도 개선



진행해보고 싶은 것 :
1. 가설수립,, 브라질 특성을 분석 해서 초입에 대립가설을 세운 후에 귀무가설을 기각할 수 있는지 생각해보기 
2. 변수 간에 상관분석이 가능할지 판단하기(지역과 잘 팔리는 상품의 연관성?
            
